# Function Calling on Databricks: A Comprehensive Guide for AI Applications

Function calling has emerged as a powerful capability for controlling Large Language Model (LLM) outputs, enabling them to generate more structured and reliable responses. Databricks has implemented OpenAI-compatible function calling as part of their Foundation Model APIs and external model serving endpoints. Let's explore how this feature works and how you can leverage it in your generative AI applications.

## Understanding Function Calling

Function calling provides developers with a mechanism to guide LLMs toward producing structured outputs. Rather than having the model execute functions directly, it generates JSON objects that describe how specific functions should be called, allowing developers to maintain control over the actual function execution in their code.

### The Basic Workflow

The function calling process on Databricks follows a straightforward sequence:

1. **Initial Query**: You call the model with your query and define a set of available functions using the `tools` parameter
2. **Model Decision**: The model analyzes the query and decides whether to call any defined functions
3. **JSON Generation**: If a function call is warranted, the model generates a JSON object adhering to your custom schema
4. **Function Execution**: Your code parses the JSON and executes the appropriate function with the provided arguments
5. **Response Completion**: The model receives the function results and generates a final summary for the user

## Practical Use Cases

Function calling shines in several scenarios:

- **Intelligent Assistants**: Build assistants that can interact with external APIs, such as `send_email()` or `get_current_weather()`
- **Natural Language to API Translation**: Convert user statements like "Who are my top customers?" into structured API calls with proper parameters
- **Data Processing**: Transform unstructured data into structured formats (though Databricks recommends their structured outputs feature for batch inference tasks)

## Supported Models

Databricks supports function calling across a wide range of models through two main serving features:

### Foundation Model APIs
- GPT-5 series (including GPT-5.1, GPT-5 mini, and GPT-5 nano)
- Gemini models (3 Pro Preview, 2.5 Pro, 2.5 Flash)
- Claude models (Sonnet-4.5, Sonnet-4, Opus-4.1, 3.7-Sonnet)
- Meta Llama models (4 Maverick, 3.3-70B-Instruct, 3.1 variants)
- GPT OSS models (20B, 120B)
- Gemma-3-12B
- Qwen3-Next 80B A3B Instruct (Beta)

### External Models
Multiple versions of GPT-4o, Claude-3 series models (through both Anthropic and AWS Bedrock providers), with some supporting advanced features like Computer Use (beta).

## Implementation Example

Here's a practical example using the OpenAI SDK with Databricks:

```python
import os
import json
from openai import OpenAI

# Initialize client
client = OpenAI(
    api_key=os.environ.get('YOUR_DATABRICKS_TOKEN'),
    base_url=os.environ.get('YOUR_DATABRICKS_BASE_URL')
)

# Define your function schema
tools = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            }
        }
    }
}]

# Make the API call
messages = [{"role": "user", "content": "What is the current temperature of Chicago?"}]
response = client.chat.completions.create(
    model="databricks-meta-llama-3-3-70b-instruct",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
```

## Controlling Function Behavior

The `tool_choice` parameter offers flexibility in how functions are called:

- **"auto"** (default): The model decides whether and which functions to call
- **"required"**: Forces the model to call one or more functions
- **Specific function**: Direct the model to call only a particular function
- **"none"**: Disable function calling entirely

## Important Considerations

### JSON Schema Guidelines

For optimal results, keep your function definitions simple. Databricks supports a subset of JSON schema specifications with these key limitations:

- Maximum of 16 keys in your JSON schema
- No support for regular expressions (`pattern`)
- Avoid complex schema compositions (`anyOf`, `oneOf`, `allOf`)
- Simpler, flatter schemas produce better quality outputs

### Token Usage and Costs

Be aware that function calling uses prompt injection and enhancement techniques, which increase both input and output token consumption. The more tools you define, the higher your input token count—and your costs.

### Current Limitations

During the Public Preview phase:
- The feature is optimized for single-turn function calling
- Parallel function calling is not supported
- Maximum of 32 functions can be defined in `tools`
- For multi-turn scenarios, Claude models are recommended
- Provisioned throughput only works on new endpoints

## Getting Started

Databricks provides a comprehensive notebook with detailed function calling examples to help you get started. The feature is available through Foundation Model APIs in specific regions, with region availability varying by model and serving type.

Function calling on Databricks represents a significant step forward in building more reliable, structured AI applications. By giving developers greater control over LLM outputs while maintaining the flexibility of natural language interaction, it opens new possibilities for creating sophisticated AI-powered tools and assistants.

---

*For the most up-to-date information on supported models, regions, and features, consult the official Databricks documentation.*

In [0]:
!pip install -qq databricks-sdk[openai]>=0.35.0

In [0]:
import os
import json
from openai import OpenAI

DATABRICKS_TOKEN = ""
DATABRICKS_BASE_URL = "https://fe-vm-agentic-ai.cloud.databricks.com/serving-endpoints"

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url=DATABRICKS_BASE_URL
)

tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string", 
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        }
      }
    }
  }
]

messages = [{"role": "user", "content": "What is the current temperature of Chicago?"}]

response = client.chat.completions.create(
    model="databricks-meta-llama-3-3-70b-instruct",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

print(json.dumps(response.choices[0].message.model_dump()['tool_calls'], indent=2))

In [0]:
response

In [0]:
resp = response.choices[0].message
resp

In [0]:
if resp.tool_calls:
    call_args = [c.function.arguments for c in resp.tool_calls]
    if len(call_args) == 1:
        print(call_args[0])
    print(call_args)

In [0]:
print(resp.content)

In [0]:
print(resp)

In [0]:
messages = [{"role": "user", "content": "What is the capital of France?"}]

response = client.chat.completions.create(
    model="databricks-meta-llama-3-3-70b-instruct",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

If the model responding without using any tools

In [0]:
resp = response.choices[0].message
if resp.tool_calls:
    call_args = [c.function.arguments for c in resp.tool_calls]
    if len(call_args) == 1:
        print(call_args[0])
    print(call_args)

In [0]:
print(resp.content)